This Notebook will be dedicated to parsing the Play-By-Play data from RetroSheets. This will allow us to gather data on a per-player basis to give us many more features with the hope of deriving more meaningful results from the models we train. The goal for this parser will to give us a boxscore-like DataFrame for each game from 2014-2019. There will be 18 batters for both NL and AL games. For the sake of simplicity, pitching stats are not included in the PBP for now, but this can easily be changed for the future. Let's walk through how we'll parse this data.

We will go through each play of each game, and update statistics for each player after each play. 

The fields of each player will be:

- (Home/Visitor) Player i:
    - player id
    - hits
    - singles
    - doubles
    - triples
    - walks
    - bunts
    - sacrifice hits
    - sacrifice flys
    - RBIs
    - at bats
    - number of stolen bases
    - number times caught stealing
    - number times picked off
    - number of errors
    
Therefore each player has 15 fields associated with them, and since there will be 18 batters, there will be a total of 270 columns of batter data. An example of one of these columns is "Visiting Player i Errors" where "i" is the ith player on the visiting team, which is determined by the "batting" flag in the event file and "Errors" is determined by summing the number of errors that player committed over the course of a single game.

In addition to the 270 batter data columns, there will be one column for the game ID, which be used to get the stats on a per-game basis.

In [34]:
import pandas as pd

column_labels = ["game id",
"visiting team",
"inning",
"batting team",
"outs",
"balls",
"strikes",
"pitch sequence",
"vis score",
"home score",
"batter",
"batter hand",
"res batter",
"res batter hand",
"pitcher",
"pitcher hand",
"res pitcher",
"res pitcher hand",
"catcher",
"first base",
"second base",
"third base",
"shortstop",
"left field",
"center field",
"right field",
"first runner",
"second runner",
"third runner",
"event text",
"leadoff flag",
"pinchhit flag",
"defensive position",
"lineup position",
"event type",
"batter event flag",
"ab flag",
"hit value",
"SH flag",
"SF flag",
"outs on play",
"double play flag",
"triple play flag",
"RBI on play",
"wild pitch flag",
"passed ball flag",
"fielded by",
"batted ball type",
"bunt flag",
"foul flag",
"hit location",
"num errors",
"1st error player",
"1st error type",
"2nd error player",
"2nd error type",
"3rd error player",
"3rd error type",
"batter dest",
"runner on 1st dest",
"runner on 2nd dest",
"runner on 3rd dest",
"play on batter",
"play on runner on 1st",
"play on runner on 2nd",
"play on runner on 3rd",
"SB for runner on 1st flag",
"SB for runner on 2nd flag",
"SB for runner on 3rd flag",
"CS for runner on 1st flag",
"CS for runner on 2nd flag",
"CS for runner on 3rd flag",
"PO for runner on 1st flag",
"PO for runner on 2nd flag",
"PO for runner on 3rd flag",
"Responsible pitcher for runner on 1st",
"Responsible pitcher for runner on 2nd",
"Responsible pitcher for runner on 3rd",
"New Game Flag",
"End Game Flag",
"Pinch-runner on 1st",
"Pinch-runner on 2nd",
"Pinch-runner on 3rd",
"Runner removed for pinch-runner on 1st",
"Runner removed for pinch-runner on 2nd",
"Runner removed for pinch-runner on 3rd",
"Batter removed for pinch-hitter",
"Position of batter removed for pinch-hitter",
"Fielder with First Putout (0 if none)",
"Fielder with Second Putout (0 if none)",
"Fielder with Third Putout (0 if none)",
"Fielder with First Assist (0 if none)",
"Fielder with Second Assist (0 if none)",
"Fielder with Third Assist (0 if none)",
"Fielder with Fourth Assist (0 if none)",
"Fielder with Fifth Assist (0 if none)",
"event num",]

mets_2014 = pd.read_csv('datasets/retro_sheets_pbp_filtered/2014NYN.EVN', names=column_labels)

mets_2014.head()

,game id,visiting team,inning,batting team,outs,balls,strikes,pitch sequence,vis score,home score,...,Position of batter removed for pinch-hitter,Fielder with First Putout (0 if none),Fielder with Second Putout (0 if none),Fielder with Third Putout (0 if none),Fielder with First Assist (0 if none),Fielder with Second Assist (0 if none),Fielder with Third Assist (0 if none),Fielder with Fourth Assist (0 if none),Fielder with Fifth Assist (0 if none),event num
0,NYN201403310,WAS,1,0,0,2,2,BCSFBFFX,0,0,...,0,5,0,0,0,0,0,0,0,1
1,NYN201403310,WAS,1,0,1,1,1,BFX,0,0,...,0,0,0,0,0,0,0,0,0,2
2,NYN201403310,WAS,1,0,1,2,2,BBCC1S,0,0,...,0,2,0,0,0,0,0,0,0,3
3,NYN201403310,WAS,1,0,2,3,0,BBBX,0,0,...,0,3,0,0,5,0,0,0,0,4
4,NYN201403310,WAS,1,1,0,2,2,CCBBS,0,0,...,0,3,0,0,2,0,0,0,0,5


In [35]:
mets_2014.shape

(6263, 97)

In [111]:
# Let's start by getting Starting Lineups
import os

gl_path = 'datasets/retro_sheet_gls/'
sl_path = 'datasets/starting_lineups/'
ignore = ['.ipynb_checkpoints']

if 'starting_lineups' not in os.listdir('datasets/'):
    os.mkdir('datasets/starting_lineups')

for file in os.listdir(gl_path):
    if file not in ignore:
        gl = pd.read_csv(gl_path + file)
        gl['Game ID'] = gl['Home Team'].str.cat(
                gl['Date'].astype('string').str.cat(
                gl['Number of game'].astype('string')))
        starting_lineups = gl.filter(items=['Game ID', 'Visiting Team Score', 'Home Team Score'] + 
                                     [f'Visiting Team Player {i+1} ID' for i in range(9)] +
                                     ['Visiting Team Starting Pitcher ID'] +
                                     [f'Home Team Player {i+1} ID' for i in range(9)] +
                                     ['Home Team Starting Pitcher ID'],
                                    axis=1)
        starting_lineups.to_csv(sl_path + 'SL' + file[2:6] + '.csv')

In [37]:
# get all play-by-play data
pbp_data_path = 'datasets/retro_sheets_pbp_filtered/'
pbp_data = []
for file in os.listdir(pbp_data_path):
    if '.EVN' in file or '.EVA' in file:
        pbp_data.append(pd.read_csv(pbp_data_path + file, names=column_labels))
pbp_data = pd.concat(pbp_data, ignore_index=True)

pbp_data

,game id,visiting team,inning,batting team,outs,balls,strikes,pitch sequence,vis score,home score,...,Position of batter removed for pinch-hitter,Fielder with First Putout (0 if none),Fielder with Second Putout (0 if none),Fielder with Third Putout (0 if none),Fielder with First Assist (0 if none),Fielder with Second Assist (0 if none),Fielder with Third Assist (0 if none),Fielder with Fourth Assist (0 if none),Fielder with Fifth Assist (0 if none),event num
0,CHN201704100,LAN,1,0,0,2,2,BCCFBC,0,0,...,0,2,0,0,0,0,0,0,0,1
1,CHN201704100,LAN,1,0,1,3,2,FFBBBX,0,0,...,0,3,0,0,0,0,0,0,0,2
2,CHN201704100,LAN,1,0,2,2,1,BCBX,0,0,...,0,9,0,0,0,0,0,0,0,3
3,CHN201704100,LAN,1,1,0,0,1,SX,0,0,...,0,7,0,0,0,0,0,0,0,4
4,CHN201704100,LAN,1,1,1,3,1,BBSBB,0,0,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144617,LAN201810010,COL,9,0,0,0,0,...X,0,5,...,0,0,0,0,0,0,0,0,0,67
1144618,LAN201810010,COL,9,0,0,3,2,FFBFBFFBX,1,5,...,0,0,0,0,0,0,0,0,0,68
1144619,LAN201810010,COL,9,0,0,0,0,X,2,5,...,0,3,0,0,0,0,0,0,0,69
1144620,LAN201810010,COL,9,0,1,2,2,CCBBFC,2,5,...,0,2,0,0,0,0,0,0,0,70


In [126]:
# get all starting lineups
sl_path = 'datasets/starting_lineups/'
ignore = ['.ipynb_checkpoints']

sl = list()

for file_num in range(2014, 2020, 1):
    sl.append(pd.read_csv(f'datasets/starting_lineups/SL{file_num}.csv'))  
        
starting_lineups = pd.concat(sl, ignore_index=True).drop(columns='Unnamed: 0')

starting_lineups

Game ID  Visiting Team Score  Home Team Score  \
0      ARI201403220                    3                1   
1      ARI201403230                    7                5   
2      SDN201403300                    1                3   
3      ANA201403310                   10                3   
4      BAL201403310                    1                2   
...             ...                  ...              ...   
14572  CHA201909290                    3                5   
14573  KCA201909290                    4                5   
14574  SEA201909290                    1                3   
14575  TEX201909290                    1                6   
14576  TOR201909290                    3                8   

      Visiting Team Player 1 ID Visiting Team Player 2 ID  \
0                      puigy001                  turnj001   
1                      gordd002                  puigy001   
2                      crawc002                  puigy001   
3                      almoa001                  millb002   
4                      navad002                  pedrd001   
...                         ...                       ...   
14572                  reyev001                  mercj002   
14573                  wadel001                  polaj001   
14574                  semim001                  profj001   
14575                  lemad001                  judga001   
14576                  diazy001                  phamt001   

      Visiting Team Player 3 ID Visiting Team Player 4 ID  \
0                      ramih003                  gonza003   
1                      ramih003                  gonza003   
2                      ramih003                  gonza003   
3                      canor001                  smoaj001   
4                      ortid001                  napom001   
...                         ...                       ...   
14572                  cabrm001                  hickj001   
14573                  sanom001                  cronc002   
14574                  piscs001                  davik003   
14575                  gardb001                  stanm004   
14576                  choij001                  garca003   

      Visiting Team Player 5 ID Visiting Team Player 6 ID  \
0                      vanss001                  uribj002   
1                      ethia001                  ellia001   
2                      ethia001                  uribj002   
3                      morrl001                  seagk001   
4                      carpm001                  sizeg001   
...                         ...                       ...   
14572                  rodrr009                  stewc002   
14573                  cavej001                  schoj001   
14574                  brows003                  phegj001   
14575                  torrg001                  sancg002   
14576                  kierk001                  robed004   

      Visiting Team Player 7 ID Visiting Team Player 8 ID  \
0                      ethia001                  ellia001   
1                      baxtm001                  uribj002   
2                      ellia001                  gordd002   
3                      saunm001                  ackld001   
4                      bogax001                  piera001   
...                         ...                       ...   
14572                  demet001                  greig001   
14573                  castj006                  torrr001   
14574                  neuss001                  barrf001   
14575                  gregd001                  urshg001   
14576                  duffm002                  zunim001   

      Visiting Team Player 9 ID  ... Home Team Player 1 ID  \
0                      kersc001  ...              polla001   
1                      ryu-h001  ...              polla001   
2                      ryu-h001  ...              cabre001   
3                      zunim001  ...              calhk001   
4                      middw001  ... 

In [13]:
# calculate a batters stats for a single game
def calculate_stats(data, player_id):
    stats = dict()
    # calculate batting stats
    h, s, d, t, hr, w = 0, 0, 0, 0, 0, 0
    for event in data[data['batter'] == player_id]['event type']:
        if event == 20:
            s, h = s+1, h+1
        elif event == 21:
            d, h = d+1, h+1
        elif event == 22:
            t, h = t+1, h+1
        elif event == 23:
            hr, h = hr+1, h+1
        elif event == 14 or event == 15:
            w += 1
    # calculate at-bats, sh, sh, RBIs and bunts
    tf = ['ab flag', 'SH flag', 'SF flag', 'bunt flag']
    tf_val = [0,0,0,0]
    for label_index in range(len(tf)):
        tf_val[label_index] += data[(data['batter'] == player_id) & (data[tf[label_index]] == 'T')].shape[0]
    ab, sh, sf, b = tf_val
    rbi = sum(data[data['batter'] == player_id]['RBI on play'])
    # calculate base running stats
    sb, cs, po = 0, 0, 0
    sb += data[(player_id == data['first runner']) & ('T' == data['SB for runner on 1st flag'])].shape[0]
    cs += data[(player_id == data['first runner']) & ('T' == data['CS for runner on 1st flag'])].shape[0]
    po += data[(player_id == data['first runner']) & ('T' == data['PO for runner on 1st flag'])].shape[0]

    sb += data[(player_id == data['second runner']) & ('T' == data['SB for runner on 2nd flag'])].shape[0]
    cs += data[(player_id == data['second runner']) & ('T' == data['CS for runner on 2nd flag'])].shape[0]
    po += data[(player_id == data['second runner']) & ('T' == data['PO for runner on 2nd flag'])].shape[0]

    sb += data[(player_id == data['third runner']) & ('T' == data['SB for runner on 3rd flag'])].shape[0]
    cs += data[(player_id == data['third runner']) & ('T' == data['CS for runner on 3rd flag'])].shape[0]
    po += data[(player_id == data['third runner']) & ('T' == data['PO for runner on 3rd flag'])].shape[0]
    # calculate errors
    err = 0
    err += data[(player_id == data['1st error player'])].shape[0]
    err += data[(player_id == data['2nd error player'])].shape[0]
    err += data[(player_id == data['3rd error player'])].shape[0]
    # fill new dictionary with stats
    stats[f'ID'] = [player_id]
    stats[f'Hits'] = [h]
    stats[f'Singles'] = [s]
    stats[f'Doubles'] = [d]
    stats[f'Triples'] = [t]
    stats[f'Home Runs'] = [hr]
    stats[f'Walks'] = [w]
    stats[f'Bunts'] = [b]
    stats[f'Sacrifice Bunts'] = [sh]
    stats[f'Sacrifice Flies'] = [sf]
    stats[f'RBIs'] = [rbi]
    stats[f'At-bats'] = [ab]
    stats[f'Stolen Bases'] = [sb]
    stats[f'Caught Stealing'] = [cs]
    stats[f'Picked Off'] = [po]
    stats[f'Errors'] = [err]
    
    # return stats
    return stats

In [16]:
def pbp_parser(pbp_data):
    # get column labels for new play-by-play dataframe
    defaults = ['Game ID']
    player_stats = ['Hits', 'Singles',
                    'Doubles', 'Triples', 'Home Runs', 
                    'Walks', 'Bunts', 'Sacrifice Bunts', 
                    'Sacrifice Flies', 'RBIs', 'At-bats', 
                    'Stolen Bases', 'Caught Stealing', 
                    'Picked Off', 'Errors']
    pbp_labels = defaults + player_stats
    
    # make a directory for individual player stats
    players_dir = 'datasets/player_stats/'
    if 'player_stats' not in os.listdir('datasets/'):
        os.mkdir(players_dir)
    
    # loop thru all pbp
    for i in range(pbp_data.shape[0]):
        batter_id = pbp_data.loc[i, 'batter']
        
        batter_filename = f'{batter_id}.csv'
        
        already_seen = set() # set to track which games we've already calculated stats for
        
        # check if we've already done this batter
        if batter_filename in os.listdir(players_dir):
            continue
        else:
            batter_stats = pd.DataFrame(columns=pbp_labels)
            
            batter_pbp = pbp_data[pbp_data['batter'] == batter_id].reset_index()
            for j in range(batter_pbp.shape[0]):
                # get game ID
                game_id = batter_pbp.loc[j, 'game id']
                # have we already done this game?
                if game_id in already_seen:
                    continue
                else:
                    # get game's play by play
                    current_game_pbp = batter_pbp[batter_pbp['game id'] == game_id]
                    # append this games stats to end of data frame
                    current_game_stats = pd.DataFrame(data=calculate_stats(data=current_game_pbp, player_id=batter_id))
                    current_game_stats['Game ID'] = game_id
                    batter_stats = pd.concat([batter_stats, current_game_stats])
                    # add to set of already seen games so no duplicates
                    already_seen.add(game_id)
                    
            # write batter's stats to csv
            batter_stats.to_csv(players_dir + batter_filename)
        
    return

In [17]:
pbp_parser(pbp_data)

In [18]:
batter = pbp_data.loc[0,'batter']
filename = f'{batter}.csv'
hey = pd.read_csv('datasets/player_stats/' + filename)
hey.head()

,Unnamed: 0,Game ID,Hits,Singles,Doubles,Triples,Home Runs,Walks,Bunts,Sacrifice Bunts,Sacrifice Flies,RBIs,At-bats,Stolen Bases,Caught Stealing,Picked Off,Errors,ID
0,0,CHN201704100,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,forsl001
1,0,CHN201704120,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,forsl001
2,0,CHN201704130,1,1,0,0,0,0,0,0,0,0,4,0,0,0,0,forsl001
3,0,WAS201709160,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,forsl001
4,0,WAS201709170,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,forsl001


In [19]:
hey.shape

(716, 18)

In [21]:
dir = 'datasets/player_stats/group_'
os.mkdir(dir + '1')
os.mkdir(dir + '2')
os.mkdir(dir + '3')

In [22]:
#os.mkdir('datasets/player_stats/group_1')
#os.mkdir('datasets/player_stats/group_2')
#os.mkdir('datasets/player_stats/group_3')

directory = 1
count = 0
ignore = ['group_1', 'group_2', 'group_3']
for file in os.listdir('datasets/player_stats/'):
    if file not in ignore:
        os.rename(f'datasets/player_stats/{file}', f'datasets/player_stats/group_{directory}/{file}')
        count += 1
        if count == 1000:
            directory +=1
            count = 0

In [23]:
len(os.listdir('datasets/player_stats/'))

3

In [24]:
print(len(os.listdir('datasets/player_stats/group_1')))
print(len(os.listdir('datasets/player_stats/group_2')))
print(len(os.listdir('datasets/player_stats/group_3')))

1000
1000
76


In [140]:
def calculate_pitcher_stats(data, pitcher_id):
    # balls and strikes need to be updated - incorrect - parse 'pitch sequence'
    balls = data[(data['pitcher'] == pitcher_id) & (data['batter event flag'] == 'T')]['balls'].sum()
    strikes = data[(data['pitcher'] == pitcher_id) & (data['batter event flag'] == 'T')]['strikes'].sum()
    strikeouts, pickoff_errs, pickoffs, wild_pitches, balks, walks, int_walks, hbp = 0, 0, 0, 0, 0, 0, 0, 0
    for event in data[data['pitcher'] == pitcher_id]['event type']:
        if event == 3: 
            strikeouts += 1
        elif event == 7:
            pickoff_errs += 1
        elif event == 8:
            pickoffs += 1
        elif event == 9:
            wild_pitches += 1
        elif event == 11:
            balks += 1
        elif event == 14:
            walks += 1
        elif event == 15:
            int_walks += 1
        elif event == 16:
            hbp += 1
    er = data[(data['pitcher'] == pitcher_id) & (data['batter dest'] == 4)].shape[0]
    er += data[(data['Responsible pitcher for runner on 1st'] == pitcher_id) & (data['runner on 1st dest'] == 4)].shape[0]
    er += data[(data['Responsible pitcher for runner on 2nd'] == pitcher_id) & (data['runner on 2nd dest'] == 4)].shape[0]
    er += data[(data['Responsible pitcher for runner on 3rd'] == pitcher_id) &(data['runner on 3rd dest'] == 4)].shape[0]
    ip = data[data['pitcher'] == pitcher_id]['inning'].max()
    era = er * 9 / ip
    
    stats = dict()
    stats['Balls'] = [balls]
    stats['Strikes'] = [strikes]
    stats['Strikeouts'] = [strikeouts]
    stats['Pickoff Errors'] = [pickoff_errs]
    stats['Pickoffs'] = [pickoffs]
    stats['Wild Pitches'] = [wild_pitches]
    stats['Balks'] = [balks]
    stats['Walks'] = [walks]
    stats['Intentional Walks'] = [int_walks]
    stats['Hit by Pitch'] = [hbp]
    stats['Earned Runs'] = [er]
    stats['Innings Pitched'] = [ip]
    stats['ERA'] = [era]
    
    return stats
    

In [121]:
def pbp_pitcher_parser(pbp_data, starting_lineups):
    # get column labels for new play-by-play dataframe
    defaults = ['Game ID']
    pitcher_stats = ['Balls', 'Strikes',
                    'Strikeouts', 'Pickoff Errors', 'Pickoffs', 
                    'Wild Pitches', 'Balks', 'Walks', 
                    'Intentional Walks', 'Hit By Pitch', 
                    'Earned Runs', 'Innings Pitched', 
                    'ERA',]
    pitcher_labels = defaults + pitcher_stats
    
    # make a directory for individual player stats
    pitcher_dir = 'datasets/pitcher_stats/'
    if 'pitcher_stats' not in os.listdir('datasets/'):
        os.mkdir(pitcher_dir)
        
    # loop thru all starting lineups
    for i in range(starting_lineups.shape[0]):
        v_label = 'Visiting Team Starting Pitcher ID'
        h_label = 'Home Team Starting Pitcher ID'
        v_pitcher_id = starting_lineups.loc[i, v_label]
        h_pitcher_id = starting_lineups.loc[i, h_label]
        
        # have we calculate stats for this pitcher?
        for pitcher_id in [v_pitcher_id, h_pitcher_id]:
            pitcher_filename = f'{pitcher_id}.csv'
        
            # check if we've already done this pitcher
            if pitcher_filename in os.listdir(pitcher_dir):
                continue
            else:
                pitcher_stats = pd.DataFrame(columns=pitcher_labels)
            
                for game_id in starting_lineups[(starting_lineups[v_label] == pitcher_id) | (starting_lineups[h_label] == pitcher_id)]['Game ID']:
                    # get game's play by play
                    current_game_pbp = pbp_data[pbp_data['game id'] == game_id]
                    # append this games stats to end of data frame
                    current_game_stats = pd.DataFrame(data=calculate_pitcher_stats(data=current_game_pbp, pitcher_id=pitcher_id))
                    current_game_stats['Game ID'] = game_id
                    pitcher_stats = pd.concat([pitcher_stats, current_game_stats])
                    
                # write pitcher's stats to csv
                pitcher_stats.to_csv(pitcher_dir + pitcher_filename)
        
    return

In [106]:
game_id = pbp_data.loc[0, 'game id']
pitcher_id = pbp_data.loc[0, 'pitcher']
pitcher_data = pbp_data[pbp_data['game id'] == game_id]
pitcher_data['runner on 1st dest']

0     0
1     0
2     0
3     0
4     0
     ..
73    1
74    0
75    2
76    0
77    0
Name: runner on 1st dest, Length: 78, dtype: int64

In [73]:
pitcher_data[pitcher_data['pitcher'] == pitcher_id]['inning'].max()

6

In [163]:
pbp_pitcher_parser(pbp_data, starting_lineups)

In [125]:
starting_lineups.shape

(2429, 21)

In [133]:
game_id = starting_lineups.loc[0, 'Game ID']
pitcher_id = starting_lineups.loc[0, 'Visiting Team Starting Pitcher ID']
pitcher_id

'kersc001'

In [142]:
game_pbp = pbp_data[pbp_data['game id'] == game_id]

game_pbp

,game id,visiting team,inning,batting team,outs,balls,strikes,pitch sequence,vis score,home score,...,Position of batter removed for pinch-hitter,Fielder with First Putout (0 if none),Fielder with Second Putout (0 if none),Fielder with Third Putout (0 if none),Fielder with First Assist (0 if none),Fielder with Second Assist (0 if none),Fielder with Third Assist (0 if none),Fielder with Fourth Assist (0 if none),Fielder with Fifth Assist (0 if none),event num
833135,ARI201403220,LAN,1,0,0,0,2,CFS,0,0,...,0,3,0,0,2,0,0,0,0,1
833136,ARI201403220,LAN,1,0,1,2,2,BBFSC,0,0,...,0,2,0,0,0,0,0,0,0,2
833137,ARI201403220,LAN,1,0,2,2,2,SBSBX,0,0,...,0,3,0,0,4,0,0,0,0,3
833138,ARI201403220,LAN,1,1,0,0,2,CSC,0,0,...,0,2,0,0,0,0,0,0,0,4
833139,ARI201403220,LAN,1,1,1,1,2,BCSX,0,0,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833203,ARI201403220,LAN,9,0,2,0,0,X,3,1,...,0,8,0,0,0,0,0,0,0,69
833204,ARI201403220,LAN,9,1,0,0,2,.CSS,3,1,...,0,2,0,0,0,0,0,0,0,70
833205,ARI201403220,LAN,9,1,1,1,2,FBSX,3,1,...,0,3,0,0,6,0,0,0,0,71
833206,ARI201403220,LAN,9,1,2,3,2,CBBBFB,3,1,...,0,0,0,0,0,0,0,0,0,72


In [162]:
pitcher_pbp = game_pbp[(game_pbp['pitcher'] == pitcher_id)]

pitcher_pbp

,game id,visiting team,inning,batting team,outs,balls,strikes,pitch sequence,vis score,home score,...,Position of batter removed for pinch-hitter,Fielder with First Putout (0 if none),Fielder with Second Putout (0 if none),Fielder with Third Putout (0 if none),Fielder with First Assist (0 if none),Fielder with Second Assist (0 if none),Fielder with Third Assist (0 if none),Fielder with Fourth Assist (0 if none),Fielder with Fifth Assist (0 if none),event num
833138,ARI201403220,LAN,1,1,0,0,2,CSC,0,0,...,0,2,0,0,0,0,0,0,0,4
833139,ARI201403220,LAN,1,1,1,1,2,BCSX,0,0,...,0,0,0,0,0,0,0,0,0,5
833140,ARI201403220,LAN,1,1,1,1,0,*BX,0,0,...,0,0,0,0,0,0,0,0,0,6
833141,ARI201403220,LAN,1,1,1,1,1,C*BX,0,0,...,0,3,0,0,1,0,0,0,0,7
833142,ARI201403220,LAN,1,1,2,2,1,BBCX,0,0,...,0,3,0,0,6,0,0,0,0,8
833148,ARI201403220,LAN,2,1,0,1,1,CBX,1,0,...,0,7,0,0,0,0,0,0,0,14
833149,ARI201403220,LAN,2,1,1,0,2,CCFS,1,0,...,0,2,0,0,0,0,0,0,0,15
833150,ARI201403220,LAN,2,1,2,1,2,CBSS,1,0,...,0,3,0,0,2,0,0,0,0,16
833154,ARI201403220,LAN,3,1,0,2,2,BBFFFFFFX,1,0,...,0,3,0,0,4,0,0,0,0,20
833155,ARI201403220,LAN,3,1,1,2,2,BBCFX,1,0,...,0,3,0,0,5,0,0,0,0,21


In [164]:
len(os.listdir('datasets/pitcher_stats/'))

747